In [141]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [142]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [143]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,10,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,50,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,50,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,50,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,50,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,50,url)

In [144]:
# getting rid of non-ascii characters from the reviews as it breaks csv
try:
    df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
except:
    pass

In [145]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [146]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



18658
18270


In [147]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
760,Bud Light,2.9,+1.7,Anheuser-Busch InBev (AB InBev),It's one of my favorite in hot summer day. Not...,HopsChecker53
761,Abbaye des Rocs Triple Impériale,2.3,-1.6,Brasserie de l'Abbaye des Rocs,Couleur rouges trouble. Mousse décente. Nez su...,AssKicker
762,Bell's III: O Captain! My Captain! (Leaves of ...,1.8,-1.7,Bell's Brewery,12 ounce bottle - $2.79 at Total Wine & More i...,ChainGangGuy
763,Leinenkugels Snowdrift Vanilla Porter,4.9,+1.8,Jacob Leinenkugel Brewing Company (Tenth & Bla...,This is one of my favoites. Rich vanilla goodn...,LagerChecker34


In [148]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [149]:
from selenium import webdriver
import time

In [150]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [151]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 27 new beers to search for!


In [152]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [153]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

,return_string
0,Napanee Dweller On the Threshold\n🇨🇦 Dubbel • ...
1,Milltown American Pale Ale\n🇬🇧 Pale Ale - Amer...
2,Indie Alehouse 10-4-18\n🇨🇦 Saison / Farmhouse ...
3,Bellwoods Motley Cru 2017\n🇨🇦 Sour / Wild Beer...
4,Exeter County Best\n🇬🇧 Bitter - Premium / Stro...
5,Windsor & Eton Conqueror\n🇬🇧 IPA - Black / Bro...
6,Maxlrainer Leo Weisse\n🇩🇪 Weissbier - Hefeweiz...
7,Marz Cryoflayer X\n🇺🇸 IIPA DIPA - Hazy / Doubl...
8,Brick Wildflower Sour\n🇬🇧 Sour / Wild Beer - F...
9,Wieninger Weissbier Naturtrüb Premium\n🇩🇪 Weis...


In [154]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [155]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [156]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

,return_string,name_found,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number
2,Indie Alehouse 10-4-18\n🇨🇦 Saison / Farmhouse ...,Indie Alehouse 10-4-18,🇨🇦,Saison / Farmhouse / Grisette,0,-,-,3.77,3


In [157]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [158]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
22,Tesco Lager\n🇬🇧 Pale Lager - North Am. / Light...,🇬🇧,Pale Lager - North Am. / Light / Rice,3.8,3,40,2.27,22,Tesco Lager
23,Marston's Harper's Wild Bill's IPA\n🇬🇧 Pale Al...,🇬🇧,Pale Ale - American / APA,5.0,34,23,3.18,124,Marston's Harper's Wild Bill's IPA
24,Püls-Bräu Hell\n🇩🇪 Helles / Dortmunder Export ...,🇩🇪,Helles / Dortmunder Export,4.6,14,30,2.66,31,Püls-Bräu Hell
26,Hamm's\n🇺🇸 Pale Lager - North Am. / Light / Ri...,🇺🇸,Pale Lager - North Am. / Light / Rice,4.7,1,25,2.15,677,Hamm's


In [159]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 3 new beers that were not found or returned errors. They will be added to the no search list.


In [160]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)